In [98]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [99]:
url_inicial = 'http://www.revistas.unam.mx/index.php/rmcpys/issue/archive?issuesPage=7#issues'
url_root = 'http://www.revistas.unam.mx/index.php/rmcpys/issue/archive?issuesPage=7#issues'
r=requests.get(url_inicial)
#falta la ultima scrapeada 

In [100]:
soup = BeautifulSoup(r.text, 'html.parser')

In [101]:
box = soup.find('div', id='issues')
volumen=box.findAll('div', style='float: left; width: 100%;')
vol = [x.find('a').get('href')for x in volumen]
vol

['http://www.revistas.unam.mx/index.php/rmcpys/issue/view/5924',
 'http://www.revistas.unam.mx/index.php/rmcpys/issue/view/5955',
 'http://www.revistas.unam.mx/index.php/rmcpys/issue/view/5997',
 'http://www.revistas.unam.mx/index.php/rmcpys/issue/view/6008',
 'http://www.revistas.unam.mx/index.php/rmcpys/issue/view/6037',
 'http://www.revistas.unam.mx/index.php/rmcpys/issue/view/6061']

In [102]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', id='content') 
    volumen1=box1.findAll('ul', class_='menu')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [103]:
vol2

['http://www.revistas.unam.mx/index.php/rmcpys/issue/view/5924/showToc',
 'http://www.revistas.unam.mx/index.php/rmcpys/issue/view/5955/showToc',
 'http://www.revistas.unam.mx/index.php/rmcpys/issue/view/5997/showToc',
 'http://www.revistas.unam.mx/index.php/rmcpys/issue/view/6008/showToc',
 'http://www.revistas.unam.mx/index.php/rmcpys/issue/view/6037/showToc',
 'http://www.revistas.unam.mx/index.php/rmcpys/issue/view/6061/showToc']

In [104]:
vol3 =[]
for b in vol2: 
    url_inicial2=b 
    r2 = requests.get(url_inicial2)
    soup2 = BeautifulSoup(r2.text, 'html.parser')
    box2 = soup2.find('div', id='content') 
    volumen2=box2.findAll('a')
    vol2= [o.get('href')for o in volumen2]
    vol3+=vol2

In [105]:
vol3

['http://dx.doi.org/10.22201/fcpys.2448492xe.1975.81',
 'http://www.revistas.unam.mx/index.php/rmcpys/article/view/80276',
 'http://www.revistas.unam.mx/index.php/rmcpys/article/view/80276/70771',
 'http://www.revistas.unam.mx/index.php/rmcpys/article/view/80280',
 'http://www.revistas.unam.mx/index.php/rmcpys/article/view/80280/70772',
 'http://www.revistas.unam.mx/index.php/rmcpys/article/view/80294',
 'http://www.revistas.unam.mx/index.php/rmcpys/article/view/80294/70778',
 'http://www.revistas.unam.mx/index.php/rmcpys/article/view/80296',
 'http://www.revistas.unam.mx/index.php/rmcpys/article/view/80296/70779',
 'http://www.revistas.unam.mx/index.php/rmcpys/article/view/80297',
 'http://www.revistas.unam.mx/index.php/rmcpys/article/view/80297/70781',
 'http://www.revistas.unam.mx/index.php/rmcpys/article/view/80299',
 'http://www.revistas.unam.mx/index.php/rmcpys/article/view/80299/70782',
 'http://www.revistas.unam.mx/index.php/rmcpys/article/view/80300',
 'http://www.revistas.una

In [106]:
def get_url_items(sopa,url):
    box2 = soup2.find('div', id='content') 
    volumen2=box2.findAll('a')
    vol2= [o.get('href')for o in volumen2]
    return vol3

In [107]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break 

Estoy en la pagina http://www.revistas.unam.mx/index.php/rmcpys/issue/archive?issuesPage=7#issues


In [108]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

228

In [109]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [110]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Mexicana de Ciencias Políticas y Sociales'
    a='V. Ciencias Sociales'
    tem='Sociología, Ciencia Política, Relaciones Internacionales, Administración Pública y Ciencias de la Comunicación'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('div', id='articleTitle').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', id='articleAbstract').get_text(strip=True)
        content_book['Resumen']=resu.replace('Resumen','')
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item abstract').find('p').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='file').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [111]:
list_scraper=list_scraper[0:228]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [112]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Revista Mexicana de Ciencias Políticas y Sociales,V. Ciencias Sociales,"Sociología, Ciencia Política, Relaciones Inter...",None,None,None,http://dx.doi.org/10.22201/fcpys.2448492xe.197...,None
1,Revista Mexicana de Ciencias Políticas y Sociales,V. Ciencias Sociales,"Sociología, Ciencia Política, Relaciones Inter...",Presentación,"Con este número, la Dirección de la Facultad...",None,http://www.revistas.unam.mx/index.php/rmcpys/a...,http://www.revistas.unam.mx/index.php/rmcpys/a...
2,Revista Mexicana de Ciencias Políticas y Sociales,V. Ciencias Sociales,"Sociología, Ciencia Política, Relaciones Inter...",None,None,None,http://www.revistas.unam.mx/index.php/rmcpys/a...,None
3,Revista Mexicana de Ciencias Políticas y Sociales,V. Ciencias Sociales,"Sociología, Ciencia Política, Relaciones Inter...",La política exterior norteamericana en la déca...,Ningún análisis de la actual política exter...,None,http://www.revistas.unam.mx/index.php/rmcpys/a...,http://www.revistas.unam.mx/index.php/rmcpys/a...
4,Revista Mexicana de Ciencias Políticas y Sociales,V. Ciencias Sociales,"Sociología, Ciencia Política, Relaciones Inter...",None,None,None,http://www.revistas.unam.mx/index.php/rmcpys/a...,None
...,...,...,...,...,...,...,...,...
223,Revista Mexicana de Ciencias Políticas y Sociales,V. Ciencias Sociales,"Sociología, Ciencia Política, Relaciones Inter...",None,None,None,https://www.facebook.com/RMCPyS.UNAM,None
224,Revista Mexicana de Ciencias Políticas y Sociales,V. Ciencias Sociales,"Sociología, Ciencia Política, Relaciones Inter...",None,None,None,https://twitter.com/RMCPyS?lang=es,None
225,Revista Mexicana de Ciencias Políticas y Sociales,V. Ciencias Sociales,"Sociología, Ciencia Política, Relaciones Inter...",None,None,None,https://www.youtube.com/channel/UCntZxC1qW9OaY...,None
226,Revista Mexicana de Ciencias Políticas y Sociales,V. Ciencias Sociales,"Sociología, Ciencia Política, Relaciones Inter...",None,None,None,https://mx.linkedin.com/in/revista-mexicana-de...,None


In [113]:
df_catalogo.to_csv('Revista049.07.csv', index=False)